In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import folium
import gmaps
import os
import requests
import shutil

In [3]:
with open('../resources/key.txt') as f:
    token = f.read()
    
gmaps.configure(api_key = token)

In [4]:
input_folder = '../data/railways/sound_transit/'
input_file = input_folder+'LINKLine.shp'

output_folder = '../data/output_images/SeattleLLR/'
output_csv = output_folder+'SEA.csv'

update_folder = output_folder+'/set_3/'

### Import Routes

In [5]:
routes = gpd.read_file(input_file)

routes = routes.to_crs({'init' :'epsg:4269'})

routes.head()

,OBJECTID,LINK_TYPE,DESCRIPTIO,STATUS,Shape_Leng,geometry
0,68,1,Central Link,COMPLETE,73476.308611,(LINESTRING (-122.2886826905774 47.46424879456...
1,101,4,Airport Link,COMPLETE,9028.515287,LINESTRING (-122.296751670065 47.4444939187097...
2,109,2,Tacoma Link,COMPLETE,9381.691809,LINESTRING (-122.4400288855092 47.256333855845...
3,113,6,University Link,CONST,16875.317490,LINESTRING (-122.3311232117972 47.613775582999...
4,130,3,Northgate Link,CONST,22284.085571,LINESTRING (-122.3281386191352 47.705615661930...


In [5]:
routes.shape

(6, 6)

In [6]:
'''
Get all points
'''

points = []
rows = routes.index.tolist()

geom_types = routes['geometry'].tolist()
for i in rows:
    try:
        route = routes.loc[i]
        if route.geometry.geom_type == 'LineString':
            g = route.geometry
            sec_points = mapping(g)["coordinates"]
            for j in sec_points:
                tmp = (j[0],j[1])
                points.append(tmp)
        elif route.geometry.geom_type == 'MultiLineString':
            g = route.geometry
            tmp = mapping(g)["coordinates"]
            for sec_points in tmp:
                for j in sec_points:
                    tmp = (j[0],j[1])
                    points.append(tmp)
    except Exception as e:
        print("Skipped route number",i,'because',e)        
        
    
print(len(points),'points')   

5610 points


In [10]:
'''
Get points for 1 or more routes
'''

rows = [0]

points = []
for i in range(0,len(routes.OBJECTID.tolist())):
    if i in rows:
        try:
            route = routes.iloc[i]
            g = [m for m in route.geometry]
            for j in range(0,len(g)):
                sec_points = mapping(g[j])["coordinates"]
                for k in range(0,len(sec_points)):
                    points.append(sec_points[k])
        except Exception as e:
            print("Skipped route number",i,'because',e)

print(len(points),'points')

3973 points


In [6]:
'''
Get points for single route/section
'''

row = 0
section = 0

points = []
try:
    route = routes.iloc[row]
    g = [i for i in route.geometry]
    len(g)

    points = mapping(g[section])["coordinates"]
except Exception as e:
    print("Skipped route number",row,'because',e)
    
print(len(points),'points')

1018 points


In [7]:
'''
Open csv with current points
'''

curr_df = pd.read_csv(output_csv)
curr_df.head()

,Name,Longitude,Latitude,Catenary
0,-122.2688245466118_47.469195680544566,-122.268825,47.469196,1
1,-122.26892266998178_47.46699028176158,-122.268923,47.466990,1
2,-122.26915916640066_47.466252365664246,-122.269159,47.466252,1
3,-122.26961937556017_47.465774616096645,-122.269619,47.465775,1
4,-122.2697075007809_47.47314172665546,-122.269707,47.473142,1


In [8]:
'''
Get subselection of points excuding current points
'''
curr_points = list(zip(curr_df['Longitude'].tolist(),curr_df['Latitude'].tolist()))
# print(curr_points)
# print('\n')

route_points = curr_points
# []
# for point in range(0,len(points),40):
#     tmp = points[point]
#     if tmp not in curr_points:
# #         print(tmp)
#         route_points.append(tmp)

# print(len(route_points))

In [9]:
'''
Create dataframe
'''

df = pd.DataFrame(route_points, columns=['Longitude','Latitude'])

tmp = list(df.Latitude.tolist())
names = []
for i in range(0,len(tmp)):
    name = str(df.iloc[i].Longitude)+'_'+str(df.iloc[i].Latitude)
    names.append(name)

df['Name'] = names

columns = ['Name','Longitude','Latitude']
df= df[columns]

df.head()

,Name,Longitude,Latitude
0,-122.26882450000001_47.46919568,-122.268825,47.469196
1,-122.26892269999999_47.46699028,-122.268923,47.466990
2,-122.26915919999999_47.46625237,-122.269159,47.466252
3,-122.26961940000001_47.46577462,-122.269619,47.465775
4,-122.2697075_47.47314173,-122.269707,47.473142


In [30]:
'''
Export points
'''

# Update
with open(output_csv, 'a') as f:
    df.to_csv(f, header=False,index=None)

# Write new
# df.to_csv(output_csv,header=True,index=None)
# df.to_csv('tmp.csv',header=True,index=None)

In [10]:
'''
Plot markers
'''

rows = df.index.tolist()

marker_points = []
for row in rows:
    marker_points.append((df.iloc[row].Latitude,df.iloc[row].Longitude))
    
# Set map centerpoint
coords = marker_points[0]
       
# Define map
m = folium.Map(
    location = coords,
    zoom_start = 12
)
  
# Add points    
for mp in marker_points:
    folium.Marker(mp).add_to(m)
m

In [23]:
'''
Get satellite preview for image
'''

row = 1

figure_layout = {
    'width': '100%',
    'height': '800px'
}
fig = gmaps.figure(center = (df.iloc[row].Latitude,df.iloc[row].Longitude), zoom_level = 19,map_type='SATELLITE',layout=figure_layout)
fig

Figure(layout=FigureLayout(height='800px', width='100%'))

In [17]:
'''
Get all images 
'''

root = os.path.dirname(os.path.abspath('Africa.ipynb'))
image_folder = root[:-9]+update_folder[3:]
image_folder

url = 'https://maps.googleapis.com/maps/api/staticmap?'
rows = df.index.tolist()
for i in range(0,len(rows)):
    row = df.iloc[i]
    center = str(row.Latitude)+','+str(row.Longitude)
    payload = {
        'center': center, 
        'zoom': 20,
        'size': '640x640',
        'scale': 2,
        'format': 'png32',
        'maptype': 'satellite',
        'key': token
    }
    
    r = requests.get(url,params=payload,stream=True)

    name = df.iloc[i]['Name']
    filename = image_folder+name+'.png'
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f) 

In [ ]:
root = os.path.dirname()